In [1]:
import itertools
import libsbml
import numpy as np
import os
import pandas as pd
import re
import requests
import sys
import SBMLLint
import sys

sys.path.append('/Users/woosubs/Desktop/AutomateAnnotation/SBMate/')
from SBMate import sbmate
from SBMate import constants as cn
from SBMate import sbml_annotation as sa
from SBMate import dag_analyzer as da
from SBMate import uniprot_kegg_analyzer as uka

from SBMLLint.common import simple_sbml
from SBMLLint.tools import sbmllint

from urllib.request import urlopen
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
%matplotlib inline  

import networkx as nx

NOTEBOOK_DIR = os.getcwd()
DATA_DIR = os.path.join(NOTEBOOK_DIR, "../../DATA")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
files = [f for f in os.listdir(BIOMODEL_DIR) if f[-4:] == ".xml"]
len(files)

In [2]:
cn.OBJECT_ONT_MAP_FILT

{libsbml.Model: {'biological_process',
  'kegg_process',
  'molecular_function',
  'occurring entity representation'},
 libsbml.Reaction: {'biological_process',
  'kegg_process',
  'molecular_function',
  'occurring entity representation'},
 libsbml.Species: {'cellular_component',
  'chemical entity',
  'kegg_species',
  'physical entity representation',
  'uniprot'},
 libsbml.Compartment: {'cellular_component', 'physical entity representation'}}

In [3]:
one_file = 'BIOMD0000000012.xml'
k = sa.SortedSBMLAnnotation(os.path.join(BIOMODEL_DIR, one_file))
m = sa.RawSBMLAnnotation(os.path.join(BIOMODEL_DIR, one_file))

In [4]:
k.annotations

{'BIOMD0000000012': {'go': ['GO:0040029'],
  'sbo': None,
  'chebi': None,
  'kegg_species': None,
  'kegg_process': None,
  'uniprot': None,
  'object_id': 'BIOMD0000000012',
  'object_type': libsbml.Model},
 'cell': {'go': ['GO:0005623'],
  'sbo': ['SBO:0000290'],
  'chebi': None,
  'kegg_species': None,
  'kegg_process': None,
  'uniprot': None,
  'object_id': 'cell',
  'object_type': libsbml.Compartment},
 'PX': {'go': None,
  'sbo': ['SBO:0000252'],
  'chebi': None,
  'kegg_species': None,
  'kegg_process': None,
  'uniprot': ['P03023'],
  'object_id': 'PX',
  'object_type': libsbml.Species},
 'PY': {'go': None,
  'sbo': ['SBO:0000252'],
  'chebi': None,
  'kegg_species': None,
  'kegg_process': None,
  'uniprot': ['P04483'],
  'object_id': 'PY',
  'object_type': libsbml.Species},
 'PZ': {'go': None,
  'sbo': ['SBO:0000252'],
  'chebi': None,
  'kegg_species': None,
  'kegg_process': None,
  'uniprot': ['P03034'],
  'object_id': 'PZ',
  'object_type': libsbml.Species},
 'X': {'go'

In [7]:
biomd12 = sa.SortedSBMLAnnotation(file=os.path.join(BIOMODEL_DIR, one_file))
x_annotation = biomd12.annotations['X']
x_analyzer = uka.NonDAGAnalyzer(term_id=['C00046'],
                                         ontology='kegg_species',
                                         object_type=libsbml.Species)

In [9]:
x_analyzer.getConsistency('ABC')

True

In [10]:
res = [x_analyzer.getOneTermConsistency(t) for t in ['ABC']]
all(res)

True

In [12]:
x_analyzer.getOneTermConsistency('ABC')

True

In [14]:
r = requests.get(uka.ONT_TO_URL[x_analyzer.ontology]+'ABC')
r.text

'<!doctype html>\n<html><head>\n<title>KEGG GENOME: Acinetobacter baumannii ACICU</title>\n<link type="text/css" rel="stylesheet" href="/css/gn2.css?1624024372">\n<link type="text/css" rel="stylesheet" href="/css/bget.css?1624024372">\n<link rel="stylesheet" href="/css/bget.css?1624024372" type="text/css">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<link rel="stylesheet" type="text/css" href="/css/bgetm.css?1624024372" media="only screen and (max-width: 768px)">\n<style>.kcfd{display:none;}</style>\n<script>\n<!--\nvar getElementsByClassName = document.getElementsByClassName ?\n  function (cl){\n    return document.getElementsByClassName(cl);\n  } :\n  function (cl){\n    var elms = [];\n    var objs = document.getElementsByTagName(\'*\');\n    for(var i=0;i<objs.length;i++){\n      var obj = objs[i];\n      if(obj.className == cl){\n        elms.push(obj);\n      }\n    }\n    return elms;\n  };\nfunction toggleField(clk,cl,show,hide){\n  var objs = getEl

In [4]:
print(k.annotations.keys())
for one_k in k.annotations.keys():
  one_anot = k.annotations[one_k]['uniprot']
  if one_anot:
    print(k.annotations[one_k])

dict_keys(['BIOMD0000000012', 'cell', 'PX', 'PY', 'PZ', 'X', 'Y', 'Z', 'Reaction1', 'Reaction2', 'Reaction3', 'Reaction4', 'Reaction5', 'Reaction6', 'Reaction7', 'Reaction8', 'Reaction9', 'Reaction10', 'Reaction11', 'Reaction12'])
{'go': None, 'sbo': ['SBO:0000252'], 'chebi': None, 'kegg_species': None, 'kegg_process': None, 'uniprot': ['P03023'], 'object_id': 'PX', 'object_type': <class 'libsbml.Species'>}
{'go': None, 'sbo': ['SBO:0000252'], 'chebi': None, 'kegg_species': None, 'kegg_process': None, 'uniprot': ['P04483'], 'object_id': 'PY', 'object_type': <class 'libsbml.Species'>}
{'go': None, 'sbo': ['SBO:0000252'], 'chebi': None, 'kegg_species': None, 'kegg_process': None, 'uniprot': ['P03034'], 'object_id': 'PZ', 'object_type': <class 'libsbml.Species'>}


In [10]:
one_anot = k.annotations['Reaction1']
one_anot

{'go': ['GO:0006402'],
 'sbo': ['SBO:0000179'],
 'chebi': None,
 'kegg_species': None,
 'kegg_process': None,
 'uniprot': None,
 'object_id': 'Reaction1',
 'object_type': libsbml.Reaction}

In [6]:
s = uka.NonDAGAnalyzer(term_id=one_anot['uniprot'], ontology='uniprot',
               object_type=one_anot['object_type'])

In [8]:
s.getConsistency(['P03023', 'ABCDE'])

False

In [9]:
float(1)

1.0